In [4]:
import numpy as np
import pandas as pd
import xarray as xr
import zarr
import math
import glob
import pickle
import statistics
import scipy.stats as stats
from sklearn.neighbors import KernelDensity
import dask
import seaborn as sns
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

In [5]:
def get_files():
    models = glob.glob("/terra/data/cmip5/global/historical/*")
    avail={}
    for model in models:
        zg = glob.glob(str(model)+"/r1i1p1/day/2deg/zg*")
        try:
            test = zg[0]
            avail[model.split('/')[-1]] = zg
        except:
             pass
    return avail

In [6]:

files['NOAA'] = glob.glob("/home/pmarsh/NOAA_2deg/NOAA_zg/*.nc")
files['ERA5'] = glob.glob("/home/pmarsh/NOAA_2deg/ERA5_zg/*.nc")
files.pop('MIROC-ESM')

['/terra/data/cmip5/global/historical/MIROC-ESM/r1i1p1/day/2deg/zg_day_MIROC-ESM_historical_r1i1p1_19500101-20051231.nc']

In [7]:
def contourise(x):
    x = x.fillna(0)
    x = x.where((x>=limit))
    x = x/x
    return x

In [8]:
results={}
for model in files.keys():
    print(model)
    x = xr.open_mfdataset(files[model])
    if model == 'NOAA':
        x = x.rename({'hgt':'zg'})
        x = x.rename({'level':'plev'})
        x = x.sel(plev=850)
        x = x.sel(time=slice('1950','2005'))
    elif model == 'ERA5':
        x = x.rename({'level':'plev'})
        x = x.sel(plev=850)
        x = x.sel(time=slice('1979','2005'))
    else:
        x = x.sel(plev=85000)
        x = x.sel(time=slice('1950','2005'))
        x = x.load()
    x = x.sel(lat=slice(-60,0))
    x = x[['zg']]
    x = x.assign_coords(lon=(((x.lon + 180) % 360) - 180))
    with dask.config.set(**{'array.slicing.split_large_chunks': True}):
        x = x.sortby(x.lon)
    x = x.sel(lon=slice(-50,20))
    x = x.resample(time="QS-DEC").mean(dim="time",skipna=True)
    x = x.load()
    limit = np.nanquantile(x.zg.values,0.9)
    results[model]={}
    for seas in ['DJF','MAM','JJA','SON']:
        mean_seas = x.where(x.time.dt.season==str(seas)).dropna(dim='time')
        mean_seas = contourise(mean_seas).zg.fillna(0).mean(dim='time')
        results[model][seas] = mean_seas.fillna(0)
    x.close()
    
 

IPSL-CM5A-LR


KeyboardInterrupt: 

In [ ]:
scores=[]
for index in results:
    MAE=[]
    for season in ['DJF','MAM','JJA','SON']:
        ref = results['NOAA'][season]
        x = results[index][season]
        for i in ref.lat.values:
            for j in ref.lon.values:
                MAE.append(float(np.abs(ref.sel(lat=i).sel(lon=j) - x.sel(lat=i).sel(lon=j)).values))
    scores.append([index,np.mean(MAE)])


In [ ]:
resultsdf = pd.DataFrame(np.array(scores),columns=['model','score'])
resultsdf = resultsdf.sort_values('score')

In [ ]:
pickle.dump( resultsdf, open( "../HIGH_OUT/scores_2D.p", "wb" ) )

In [ ]:
resultsdf.to_csv('../HIGH_OUT/scores_2D.csv')

In [ ]:
for index in results:
    for seas in ['DJF','MAM','JJA','SON']:
        ax = plt.axes(projection=ccrs.PlateCarree())
        ax.contourf(results[index]['JJA'].lon,results[index]['JJA'].lat,results[index][str(seas)].where(results[index][str(seas)]>0),levels=100, cmap = 'plasma',alpha = 0.9,vmin=0, vmax=1,transform = ccrs.PlateCarree())
        ax.coastlines()
        ax.set_extent([-50, 20, -60, 0], ccrs.PlateCarree())
        ax.gridlines(linewidth=0.5, color='gray', alpha=0.5)
        plt.title(str(seas))
        plt.savefig("../HIGH_OUT/contour_plots/pdf/"+str(index)+"_"+str(seas)+".pdf")
        plt.savefig("../HIGH_OUT/contour_plots/png/"+str(index)+"_"+str(seas)+".png",dpi=1000)
        plt.close()

In [ ]:
for index in results:
    mean = (results[index]['DJF'] + results[index]['MAM'] +results[index]['JJA'] + results[index]['SON'])/4
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.contourf(mean.lon,mean.lat,mean.where(mean>0),levels=100, cmap = 'plasma',alpha = 0.9,vmin=0, vmax=1,transform = ccrs.PlateCarree())
    ax.coastlines()
    ax.set_extent([-50, 20, -60, 0], ccrs.PlateCarree())
    ax.gridlines(linewidth=0.5, color='gray', alpha=0.5)
    plt.title(str(seas))
    plt.savefig("../HIGH_OUT/annual_contour_plots/pdf/"+str(index)+".pdf")
    plt.savefig("../HIGH_OUT/annual_contour_plots/png/"+str(index)+".png",dpi=1000)
    plt.close()

In [13]:
ref = (results['NOAA']['DJF'] + results['NOAA']['MAM'] +results['NOAA']['JJA'] + results["NOAA"]['SON'])/4

for index in results:
    mean = (results[index]['DJF'] + results[index]['MAM'] +results[index]['JJA'] + results[index]['SON'])/4
    anom = mean - ref 
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.contourf(anom.lon,anom.lat,anom,levels=100, cmap = 'seismic_r',alpha = 1,vmin=-1, vmax=1,transform = ccrs.PlateCarree())
    ax.coastlines()
    ax.set_extent([-50, 20, -60, 0], ccrs.PlateCarree())
    ax.gridlines(linewidth=0.5, color='gray', alpha=0.2)
    plt.title(str(index))
    plt.savefig("../HIGH_OUT/anom_contour_plots/pdf/"+str(index)+".pdf")
    plt.savefig("../HIGH_OUT/anom_contour_plots/png/"+str(index)+".png",dpi=1000)
    plt.close()

In [13]:
resultsdf = pd.read_csv('../HIGH_OUT/scores_2D.csv').set_index('model')['score']

In [14]:
pickle.dump( resultsdf, open( "../HIGH_OUT/scores_2D.p", "wb" ) )

In [18]:
for index in list(resultsdf.astype(float).dropna().index):
    if index == 'NOAA':
        pass
    else:
        print('\subcaptionbox{'+str(index)+'\label{Figure 20}}{\includegraphics[width = 3.2cm,trim={1.8cm 1cm 1.8cm 1cm},clip]{../HIGH/HIGH_OUT/anom_contour_plots/pdf/'+str(index)+'}}')

\subcaptionbox{ERA5\label{Figure 20}}{\includegraphics[width = 3.2cm,trim={1.8cm 1cm 1.8cm 1cm},clip]{../HIGH/HIGH_OUT/anom_contour_plots/pdf/ERA5}}
\subcaptionbox{HadGEM2-CC\label{Figure 20}}{\includegraphics[width = 3.2cm,trim={1.8cm 1cm 1.8cm 1cm},clip]{../HIGH/HIGH_OUT/anom_contour_plots/pdf/HadGEM2-CC}}
\subcaptionbox{ACCESS1-0\label{Figure 20}}{\includegraphics[width = 3.2cm,trim={1.8cm 1cm 1.8cm 1cm},clip]{../HIGH/HIGH_OUT/anom_contour_plots/pdf/ACCESS1-0}}
\subcaptionbox{bcc-csm1-1\label{Figure 20}}{\includegraphics[width = 3.2cm,trim={1.8cm 1cm 1.8cm 1cm},clip]{../HIGH/HIGH_OUT/anom_contour_plots/pdf/bcc-csm1-1}}
\subcaptionbox{CMCC-CM\label{Figure 20}}{\includegraphics[width = 3.2cm,trim={1.8cm 1cm 1.8cm 1cm},clip]{../HIGH/HIGH_OUT/anom_contour_plots/pdf/CMCC-CM}}
\subcaptionbox{IPSL-CM5B-LR\label{Figure 20}}{\includegraphics[width = 3.2cm,trim={1.8cm 1cm 1.8cm 1cm},clip]{../HIGH/HIGH_OUT/anom_contour_plots/pdf/IPSL-CM5B-LR}}
\subcaptionbox{CNRM-CM5\label{Figure 20}}{\include